In [1]:
import sys
sys.path.append('..')

In [2]:
from tqdm import tqdm

from lib.data import load_all_invoices, load_extracted_samples
from lib.details import extract_details_from_text, find_professional_total
from lib.invoice import find_invoice_cached

In [3]:
all_invoices = load_all_invoices()
len(all_invoices)

8411

In [4]:
# load samples
samples = [sample for sample in load_extracted_samples()]
len(samples)

8411

In [5]:

invoice_dict = {}
for invoice in all_invoices:
    invoice_dict[invoice] = None

for sample in tqdm(samples):
    filename = sample["filename"][:-4] # .pdf
    
    # text = sample["text"]
    # if text == "":
    #     text = ocr_sample(sample, params, only_cache=True)
    
    invoice = find_invoice_cached(sample)

    total = None
    if sample["text"] != "":
        lines = extract_details_from_text(sample["text"])
        total = find_professional_total(lines)

    if invoice is not None and total is not None:
        invoice_dict[invoice] = total

100%|██████████| 8411/8411 [02:21<00:00, 59.39it/s] 


In [6]:
all_totals = [total for total in invoice_dict.values() if total is not None]
avg = sum(all_totals) / len(all_totals)
avg = round(avg, 2)
len(all_totals), len(all_totals) / len(samples), avg

(2117, 0.2516942099631435, 11747.75)

In [7]:
# write csv
with open("submission.csv", 'w') as f:
    f.write('Invoice Number,Total Charged\n')
    for invoice, total in invoice_dict.items():

        if total == None:
            total = avg

        f.write(f'{invoice},{total}\n')